In [1]:
import os
import shutil
import random

# Read-only dataset path
dataset_dir = "/kaggle/input/oct-images-dataset/"  # Change this to your dataset path
writable_dir = "/kaggle/working/dataset"  # Writable directory

# Copy dataset to a writable location
if not os.path.exists(writable_dir):
    shutil.copytree(dataset_dir, writable_dir)

# Define train, val, test directories
train_dir = os.path.join(writable_dir, "train")
val_dir = os.path.join(writable_dir, "val")
test_dir = os.path.join(writable_dir, "test")

# Create train, val, test directories
for directory in [train_dir, val_dir, test_dir]:
    os.makedirs(directory, exist_ok=True)

# Get all subfolders (classes)
subfolders = [f for f in os.listdir(writable_dir) if os.path.isdir(os.path.join(writable_dir, f))]

# Process each class folder separately
for subfolder in subfolders:
    subfolder_path = os.path.join(writable_dir, subfolder)

    # Get all images in this class
    all_images = [f for f in os.listdir(subfolder_path) if f.endswith(('.jpg', '.jpeg', '.png'))]
    random.shuffle(all_images)  # Shuffle before splitting

    # Define split sizes
    train_split = int(0.7 * len(all_images))
    val_split = int(0.9 * len(all_images))  # 70% train, next 20% val, last 10% test

    # Create class-specific subdirectories
    os.makedirs(os.path.join(train_dir, subfolder), exist_ok=True)
    os.makedirs(os.path.join(val_dir, subfolder), exist_ok=True)
    os.makedirs(os.path.join(test_dir, subfolder), exist_ok=True)

    # Move images to respective folders
    for i, img in enumerate(all_images):
        src_path = os.path.join(subfolder_path, img)

        if i < train_split:
            dest_path = os.path.join(train_dir, subfolder, img)
        elif i < val_split:
            dest_path = os.path.join(val_dir, subfolder, img)
        else:
            dest_path = os.path.join(test_dir, subfolder, img)

        shutil.move(src_path, dest_path)

print(f"Dataset split complete! ✅")
print(f"Train contains {len(os.listdir(train_dir))} classes.")
print(f"Validation contains {len(os.listdir(val_dir))} classes.")
print(f"Test contains {len(os.listdir(test_dir))} classes.")


Dataset split complete! ✅
Train contains 7 classes.
Validation contains 7 classes.
Test contains 7 classes.


In [2]:
# UGG16 X ResNet Hybrid Model for classification: Higher accuracy score, so kept
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.metrics import confusion_matrix, classification_report
from PIL import Image

data_dir = "/kaggle/working/dataset"
train_dir = os.path.join(data_dir, "train")
val_dir = os.path.join(data_dir, "val")
test_dir = os.path.join(data_dir, "test")

image_size = (64, 64)
batch_size = 32

train_dataset = keras.preprocessing.image_dataset_from_directory(
    train_dir, image_size=image_size, batch_size=batch_size, shuffle=True
)

val_dataset = keras.preprocessing.image_dataset_from_directory(
    val_dir, image_size=image_size, batch_size=batch_size, shuffle=False
)

test_dataset = keras.preprocessing.image_dataset_from_directory(
    test_dir, image_size=image_size, batch_size=batch_size, shuffle=False
)

class_names = train_dataset.class_names
num_classes = len(class_names)

normalization_layer = layers.Rescaling(1./255)
train_dataset = train_dataset.map(lambda x, y: ((normalization_layer(x), normalization_layer(x)), y))
val_dataset = val_dataset.map(lambda x, y: ((normalization_layer(x), normalization_layer(x)), y))
test_dataset = test_dataset.map(lambda x, y: ((normalization_layer(x), normalization_layer(x)), y))

vgg16 = keras.applications.VGG16(weights="imagenet", include_top=False, input_shape=(64, 64, 3))
vgg16.trainable = False

resnet50 = keras.applications.ResNet50(weights="imagenet", include_top=False, input_shape=(64, 64, 3))
resnet50.trainable = False
vgg16_output = layers.Flatten()(vgg16.output)
resnet50_output = layers.Flatten()(resnet50.output)

merged = layers.concatenate([vgg16_output, resnet50_output])

x = layers.Dense(512, activation="relu")(merged)
x = layers.Dropout(0.5)(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dense(128, activation="relu")(x)
output = layers.Dense(num_classes, activation="softmax")(x)

model = keras.Model(inputs=[vgg16.input, resnet50.input], outputs=output)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=20
)

model.save("Hybrid_VGG16_ResNet50.h5")
print("✅ Model saved successfully!")

test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test Accuracy: {test_acc:.4f}")

y_true, y_pred = [], []
for images, labels in test_dataset:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

print(classification_report(y_true, y_pred, target_names=class_names))

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()

def predict_image(image_path, model):
    image = Image.open(image_path).convert("RGB")
    image = image.resize((64, 64))
    image = np.array(image) / 255.0
    image = np.expand_dims(image, axis=0)
    prediction = model.predict((image, image))
    predicted_class = np.argmax(prediction)
    return class_names[predicted_class]

image_path = r"C:\Users\91995\Downloads\DRUSEN-1001666-5.jpeg"
prediction = predict_image(image_path, model)
print(f"Predicted Class: {prediction}")



Found 15301 files belonging to 7 classes.
Found 4372 files belonging to 7 classes.
Found 2188 files belonging to 7 classes.
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/20
479/479 ━━━━━━━━━━━━━━━━━━━━ 41s 57ms/step - accuracy: 0.5597 - loss: 1.1378 - val_accuracy: 0.7207 - val_loss: 0.7989
Epoch 2/20
479/479 ━━━━━━━━━━━━━━━━━━━━ 15s 32ms/step - accuracy: 0.7047 - loss: 0.8421 - val_accuracy: 0.7493 - val_loss: 0.7100
Epoch 3/20
479/479 ━━━━━━━━━━━━━━━━━━━━ 16s 33ms/step - accuracy: 0.7329 - loss: 0.7659 - val_accuracy: 0.7587 - val_loss: 0.6789
Epoch 4/20
479/479 ━━━━━━━━━━━━━━━━━━━━ 16s 33ms/step - accuracy: 0.7378 - loss: 0.7339 - val_accuracy: 0.7644 - val_loss: 0.6583
Epoch 5/20
479/479 ━━━━━━━━━━━━━━━━━━━━ 16s 34ms/step - accuracy: 0.7458 - loss: 0.7102 - val_accuracy: 0.7621 - val_loss: 0.6606
Epoch 6/20
479/479 ━━━━━━━━━━━━━━━━━━━━ 16s 34ms/step - accuracy: 0.7539 - loss: 0.6858 - val_accuracy: 0.7825 - val_loss: 0

ValueError: Number of classes, 4, does not match size of target_names, 7. Try specifying the labels parameter

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=20
)
model.save("Hybrid_VGG16_ResNet50.h5")
print("✅ Model saved successfully!")

In [ ]:
history2 = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=60
)
model.save("Hybrid_VGG16_ResNet50.h5")
print("✅ Model saved successfully!")

In [ ]:
model.save("OCT_model.h5")

**CHECKING FOR ANOTHER MODEL**

In [ ]:
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers
# import numpy as np
# import os
# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.metrics import confusion_matrix, classification_report
# from PIL import Image

# # Load dataset
# data_dir = "/kaggle/working/dataset"
# train_dir = os.path.join(data_dir, "train")
# val_dir = os.path.join(data_dir, "val")
# test_dir = os.path.join(data_dir, "test")

# image_size = (64, 64)
# batch_size = 32

# train_dataset = keras.preprocessing.image_dataset_from_directory(
#     train_dir, image_size=image_size, batch_size=batch_size, shuffle=True
# )

# val_dataset = keras.preprocessing.image_dataset_from_directory(
#     val_dir, image_size=image_size, batch_size=batch_size, shuffle=False
# )

# test_dataset = keras.preprocessing.image_dataset_from_directory(
#     test_dir, image_size=image_size, batch_size=batch_size, shuffle=False
# )

# class_names = train_dataset.class_names
# num_classes = len(class_names)

# # Apply normalization
# normalization_layer = layers.Rescaling(1./255)
# train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
# val_dataset = val_dataset.map(lambda x, y: (normalization_layer(x), y))
# test_dataset = test_dataset.map(lambda x, y: (normalization_layer(x), y))

# # Load VGG16 & ResNet50
# vgg16 = keras.applications.VGG16(weights="imagenet", include_top=False, input_shape=(64, 64, 3))
# resnet50 = keras.applications.ResNet50(weights="imagenet", include_top=False, input_shape=(64, 64, 3))

# vgg16.trainable = False
# resnet50.trainable = False

# # Define Inputs
# input_layer = keras.Input(shape=(64, 64, 3))

# # Pass input through both models
# vgg16_features = vgg16(input_layer)
# resnet50_features = resnet50(input_layer)

# # Global pooling to reduce dimensionality
# vgg16_output = layers.GlobalAveragePooling2D()(vgg16_features)
# resnet50_output = layers.GlobalAveragePooling2D()(resnet50_features)

# # Concatenate outputs
# merged = layers.concatenate([vgg16_output, resnet50_output])

# # Fully Connected Layers
# x = layers.Dense(512, activation="relu")(merged)
# x = layers.Dropout(0.5)(x)
# x = layers.Dense(256, activation="relu")(x)
# x = layers.Dense(128, activation="relu")(x)
# output = layers.Dense(num_classes, activation="softmax")(x)

# # Define Model with a single input layer
# model = keras.Model(inputs=input_layer, outputs=output)

# # Compile Model
# model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
#               loss="sparse_categorical_crossentropy",
#               metrics=["accuracy"])

# # Train Model
# history = model.fit(
#     train_dataset,
#     validation_data=val_dataset,
#     epochs=50
# )

# # Save Model
# model.save("Hybrid_VGG16_ResNet50.h5")
# print("✅ Model saved successfully!")

# # Evaluate Model
# test_loss, test_acc = model.evaluate(test_dataset)
# print(f"Test Accuracy: {test_acc:.4f}")

# # Predictions & Confusion Matrix
# y_true, y_pred = [], []
# for images, labels in test_dataset:
#     preds = model.predict(images)
#     y_true.extend(labels.numpy())
#     y_pred.extend(np.argmax(preds, axis=1))

# print(classification_report(y_true, y_pred, target_names=class_names))

# cm = confusion_matrix(y_true, y_pred)
# plt.figure(figsize=(6, 5))
# sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
# plt.xlabel("Predicted")
# plt.ylabel("True")
# plt.title("Confusion Matrix")
# plt.show()


In [ ]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=25
)